<a id='top'></a>

# CSCI 3022: Intro to Data Science - Fall 2020 Practicum 2

***

**Name**:  Nikolai Lyssogor

***

**Partner Name** (if applicable):

***

*Each individual must submit a copy of this activity - even if you worked with a partner. If you worked with a partner, you and your partner should have similar / identical work on the problems below, but your name (as the submitting student) and your partner's name should be accurately represented in the fields above.*

This practicum is due on Canvas by **10:00 PM on Saturday, December 12** (the end of your Finals Period). Your solutions to theoretical questions should be done in Markdown/MathJax directly below the associated question.  Your solutions to computational questions should include any specified Python code and results as well as written commentary on your conclusions. 

**Here are the rules:** 

1. All work, code and analysis, must be your own - either alone or with your partner. 
2. You may use your course notes, posted lecture slides, textbooks, in-class notebooks, and homework solutions as resources.  You may also search online for answers to general knowledge questions like the form of a probability distribution function or how to perform a particular operation in Python/Pandas. 
3. This is meant to be like a coding portion of your midterm exam. So, the instructional team will be much less helpful than we typically are with homework. For example, we will not check answers, help debug your code, and so on. (There is a limited exception in Problem 2B)
4. If something is left open-ended, it is because we want to see how you approach the kinds of problems you will encounter in the wild, where it will not always be clear what sort of tests/methods should be applied. Feel free to ask clarifying questions in office hours or on a private Piazza post.
5. You may **NOT** post to public message boards or other online resources asking for help.
6. You may **NOT** copy-paste solutions *from anywhere*.
7. You may **NOT** collaborate with classmates or anyone else (other than your partner).
8. In short, **your work must be your own**. It really is that simple.

Violation of the above rules will result in an immediate academic sanction (*at the very least*, you will receive a 0 on this practicum or an F in the course, depending on severity), and a trip to the Honor Code Council.

**By submitting this assignment, you agree to abide by the rules given above.**

**NOTES**: 

- You may not use late days on the practicums nor can you drop your practicum grades. 
- If you have a question for us, post it as a **PRIVATE** message on Piazza.  If we decide that the question is appropriate for the entire class, then we will add it to a Practicum clarifications thread. 
- Do **NOT** load or use any Python packages that are not available in Anaconda 3.6. 
- Some problems with code may be autograded.  If we provide a function API **do not** change it.  If we do not provide a function API then you're free to structure your code however you like. 
- Submit only this Jupyter notebook to Canvas.  Do not compress it using tar, rar, zip, etc. 
- This should go without saying, but... For any question that asks you to calculate something, you **must show all work to receive credit**. Sparse or nonexistent work will receive sparse or nonexistent credit.


---

In [352]:
from scipy import stats
from math import isnan
from patsy import dmatrices
import numpy as np 
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from calendar import month_name, different_locale

# Problem 1: Urbanization (50 pts)
It is January of 1963, and President Kennedy has hired you as the Chief Statistician for the White House!  Your first task is to analyze and report state expenditures across the country$^1$.  You feel ready for the challenge, since you studied so hard in your statistical methods class.  On day one, JFK hands you an important data set that contains information on the 48 states in the contiguous U.S. describing per capita state and local public expenditures associated with state demographic and economic characteristics in 1960$^2$ . The data set is found in the file `stateExpenditures.txt`.

You are told that you need to quantify **how per capita state and local expenditures can be explained and predicted** by:
* The economic ability index
* The percentage of the population living in a metropolitan area
* The percentage change in the population between 1950 and 1960
* The percentage of the population aged 5-19 years
* The percentage of the population over 65 years old
* Whether the state is located in the western part of the United States or not

The variables available in the data set are labeled as follows:

* EX: 		Per capita state and local public expenditures (USD)
* ECAB: 	Economic ability index, in which income, retail sales, and the value of output (manufactures, mineral, and agricultural) per capita are equally weighted
* MET: 		Percentage of population living in standard metropolitan areas
* GROW: 	Percent change in population, 1950-1960
* YOUNG: 	Percent of population aged 5-19 years
* OLD: 		Percent of population over 65 years of age
* WEST: 	Western state (1) or not (0)

Keep in mind that the president does not know how to interpret linear model output, and he wants answers in terms of things that are easily read and understood.  Therefore, when analyzing your models, *be sure your answers are friendly for a general audience, but include enough technical information that your statistics professor believes you know what you're talking about*.

$^1$ Just pretend that today's hardware and software existed in 1963 - no need to purchase a variety of vacuum tubes!

$^2$ U.S. Department of Commerce, Bureau of the Census, Government Finances in 1960, Census of Population (1960),  Census of Manufactures (1958), Statistical Abstract of the United States (1961), U.S. Department of Agriculture, Agricultural Statistics (1961), and the U.S. Department of the Interior, Minerals Yearbook (1960).

### 1A) Load Your Dataset
Load in the data set saved in "stateExpenditures.txt".  Save it into a data frame.

In [790]:
#YOUR CODE FOR 1A HERE
expDF = pd.read_csv('stateExpenditures.txt', sep='\s+')
expDF.head(10)

EX   ECAB   MET  GROW  YOUNG   OLD  WEST STATE
0  256   85.5  19.7   6.9   29.6  11.0     0    ME
1  275   94.3  17.7  14.7   26.4  11.2     0    NH
2  327   87.0   0.0   3.7   28.5  11.2     0    VT
3  297  107.5  85.2  10.2   25.1  11.1     0    MA
4  256   94.9  86.2   1.0   25.3  10.4     0    RI
5  312  121.6  77.6  25.4   25.2   9.6     0    CT
6  374  111.5  85.5  12.9   24.0  10.1     0    NY
7  257  117.9  78.9  25.5   24.8   9.2     0    NJ
8  257  103.1  77.9   7.8   25.7  10.0     0    PA
9  336  116.1  68.8  39.9   26.4   8.0     0    DE

### 1B) Consider Variable Nonlinearity
The effects of the metrolpolitan variables are highly nonlinear!  One way to approach this is to consider it as a categorical variables instead.  Convert MET to a categorical variable $METcateg$ that denotes which level MET each state is in by dividing MET up into units of 15:

Set METcateg equal to
  \begin{array}{l l}
    METcateg=1 & \quad \text{if $MET < 15$}\\
    METcateg=2 & \quad \text{if $15 \le MET < 30$}\\
    METcateg=3 & \quad \text{if $30 \le MET < 45$}\\
    METcateg=4 & \quad \text{if $45 \le MET < 60$}\\
    METcateg=5 & \quad \text{if $60 \le MET < 75$}\\
    METcateg=6 & \quad \text{if $75 \le MET$}\\    
  \end{array} 

In [791]:
#YOUR CODE FOR 1B HERE
expDFmod = expDF.rename(columns={'MET': 'METcateg'})

for i in range(0, 48):
    if expDFmod['METcateg'].loc[i] < 15 or expDFmod['METcateg'].loc[i] >= 60:
        expDFmod['METcateg'].loc[i] = 3
    elif 15 <= expDFmod['METcateg'].loc[i] < 30 or 45 <= expDFmod['METcateg'].loc[i] < 60:
        expDFmod['METcateg'].loc[i] = 2
    elif 30 <= expDFmod['METcateg'].loc[i] < 45:
        expDFmod['METcateg'].loc[i] = 1

C:\Users\lysso\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


### 1C) Visualize Your Covariates
Make pairwise scatter plots of the continuous covariates, both against each other and against the outcome (expenditures).   Does the relationship between the independent variables and the dependent variables appear to be linear?  Do there appear to be independent variables that are collinear?  You may import the package `seaborn` if desired.

In [792]:
import seaborn as sns

In [793]:
#YOUR CODE FOR 1C HERE
sns.pairplot(expDF)

Economic ability index and population growth are the two predictors for which the relationship to expenditure is most obviously linear. In the case of both of these, an increase both population and ECAB is correlated with an increase in expenditure. YOUNG is vaguely negatively correlated with expenditure, and OLD is even more vaguely positively correlated with expenditure.

Many of the independent predictors appear collinear. Most of these might be reasonably explained, such as YOUNG and OLD or GROW and ECAB. Greater YOUNG appears to have a negative effect on ECAB, while OLD has a positive one. 

### 1D) Run Your Full Model
Fit the full model using `stats.OLS` in Python.  Then in a markdown cell, write out the estimated full model, adjusted $R^2$ value, and interpret the parameters in sentences with appropriate units.

In [794]:
#YOUR CODE FOR 1D HERE
# full untouched model
X = expDF[['ECAB', 'MET', 'GROW', 'YOUNG', 'OLD', 'WEST']]
X = sm.add_constant(X) # adds constant to the array for the intercept
y = expDF['EX']
model = sm.OLS(y, X).fit()
print(X.head())

   const   ECAB   MET  GROW  YOUNG   OLD  WEST
0    1.0   85.5  19.7   6.9   29.6  11.0     0
1    1.0   94.3  17.7  14.7   26.4  11.2     0
2    1.0   87.0   0.0   3.7   28.5  11.2     0
3    1.0  107.5  85.2  10.2   25.1  11.1     0
4    1.0   94.9  86.2   1.0   25.3  10.4     0


In [795]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     EX   R-squared:                       0.599
Model:                            OLS   Adj. R-squared:                  0.541
Method:                 Least Squares   F-statistic:                     10.22
Date:                Fri, 11 Dec 2020   Prob (F-statistic):           6.63e-07
Time:                        19:15:06   Log-Likelihood:                -241.20
No. Observations:                  48   AIC:                             496.4
Df Residuals:                      41   BIC:                             509.5
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        356.1818    306.486      1.162      0.252    -262.780     975.143
ECAB           1.4185      0.430      3.298      0.002       0.550       2.287
MET           -0.6602      0.353     -1.872      0.068      -1.372       0.052
GROW           0.5716      0.425      1.345      0.186      -0.287       1.430
YOUNG         -6.6747      7.481     -0.892      0.377     -21.782       8.433
OLD           -1.8551      7.137     -0.260      0.796     -16.268      12.558
WEST          35.4723     13.771      2.576      0.014       7.661      63.284
==============================================================================
Omnibus:                        0.723   Durbin-Watson:                   2.349
Prob(Omnibus):                  0.697   Jarque-Bera (JB):                0.524
Skew:                           0.253   Prob(JB):                        0.770
Kurtosis:                       2.927   Cond. No.                     6.18e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.18e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

`stats.OLS` produced a model that when written out looks as follows:

$$
\texttt{EX}=356.18+1.42\texttt{(ECAB)}-0.66\texttt{(MET)}+0.57\texttt{(GROW)}-6.67\texttt{(YOUNG)}-1.86\texttt{(OLD)}+35.47\texttt{(WEST)}
$$

More verbosely, this model predicts per capita state and expenditure as such:
- A 1.42 USD increase in EX for a 1 point increase on the economic ability index
- A 0.66 USD decrease in EX for a 1% increase in population living in metropolitan areas
- A 0.57 USD increase in EX for a 1% change in population between 1950 and 1960
- A 6.67 USD decrease in EX for a 1 point increase in percent of the population aged 5-19 years
- A 1.86 USD decrease in EX for a 1 point increase in percent of the population aged over 65 years
- a 35.47 USD increase in EX if a state is in the west

The adjusted $R^2$ value of 0.541 means that a little over half of the variability of per capita state and expenditure is captured by the predictors we used, when adjusted for the number of predictors, since more predictors will always make the model more accurate.


### 1E) Run a Modified Full Model
Fit a "full" model that includes ECAB, METcateg, GROW, YOUNG, OLD, and WEST.  Write out the estimated model in a markdown cell afterwards.

In [796]:
#YOUR CODE FOR 1E HERE
# full model with finessed categorized MET
X_mod = expDFmod[['ECAB', 'METcateg', 'GROW', 'YOUNG', 'OLD', 'WEST']]
X_mod = sm.add_constant(X_mod) # adds constant to the array for the intercept
y_mod = expDF['EX']
model_mod = sm.OLS(y_mod, X_mod).fit()
model_mod.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     EX   R-squared:                       0.646
Model:                            OLS   Adj. R-squared:                  0.594
Method:                 Least Squares   F-statistic:                     12.46
Date:                Fri, 11 Dec 2020   Prob (F-statistic):           6.11e-08
Time:                        19:15:37   Log-Likelihood:                -238.25
No. Observations:                  48   AIC:                             490.5
Df Residuals:                      41   BIC:                             503.6
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -194.9152    231.970     -0.840      0.406    -663.387     273.557
ECAB           1.6235      0.388      4.186      0.000       0.840       2.407
METcateg      25.9802      8.504      3.055      0.004       8.806      43.155
GROW           0.3374      0.402      0.840      0.406      -0.474       1.149
YOUNG          5.6543      5.532      1.022      0.313      -5.517      16.826
OLD            8.6337      6.037      1.430      0.160      -3.559      20.826
WEST          34.8215     12.948      2.689      0.010       8.672      60.971
==============================================================================
Omnibus:                        1.621   Durbin-Watson:                   2.746
Prob(Omnibus):                  0.445   Jarque-Bera (JB):                0.818
Skew:                           0.237   Prob(JB):                        0.664
Kurtosis:                       3.429   Cond. No.                     4.52e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.52e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Estimated model for modified MET category:

$$
\texttt{EX}=354.98+1.42\texttt{(ECAB)}-10.24\texttt{(METcateg)}+0.55\texttt{(GROW)}-6.40\texttt{(YOUNG)}-1.70\texttt{(OLD)}+33.20\texttt{(WEST)}
$$


### 1F) Reduce Your Model
Perform *backwards selection* on this model.  Starting with the full model created in part 1D): remove the predictor with the highest p-value, and re-calculate the model without that predictor.  Continue this process until there are no predictors left with p-values greater than 0.05.  Write out your final estimated model. 

In [203]:
import itertools

`find_model` will try all combinations of predictors and return the ones for which all the p-values for that model are less than .05 along with the adjusted $R^2$ value.

In [797]:
#YOUR CODE FOR 1F HERE
def find_model(DF): 
    predictors = ['ECAB', 'METcateg', 'GROW', 'YOUNG', 'OLD', 'WEST']
    predictor_combs = []

    # create a list of lists of combinations of lengths 2-6
    for L in range(2, 7):
        for comb in itertools.combinations(predictors, L):
            predictor_combs.append(list(comb))

    # variables to store potential combinations and their adjusted r-squared values
    nice_combs = []
    nice_rsq_adj = []

    # model each combination of predictors
    num_combs = len(predictor_combs)
    for i in range(0, num_combs):
        flag = False
        X = DF[predictor_combs[i]]
        X = sm.add_constant(X)
        y = DF['EX']
        temp_model = sm.OLS(y, X).fit()
        
        rsquared_adj = temp_model.rsquared_adj
        pvalues = temp_model.pvalues
        num_predictors = len(pvalues)

        # check that all the p-values are < .05
        for j in range(0, num_predictors):
            if(pvalues[j] > .05):
                flag = True
                break
        if(flag == True): continue

        # if p-values look good, save this combination for later manual reference
        nice_combs.append(predictor_combs[i])
        nice_rsq_adj.append(rsquared_adj)
    
    return nice_combs, nice_rsq_adj

# note that this is using a heavily finessed version of the METcateg predictor
predictors, rsq_adj = find_model(expDFmod)
print(predictors)
print(rsq_adj)

[['ECAB', 'METcateg'], ['ECAB', 'WEST'], ['METcateg', 'WEST'], ['GROW', 'WEST'], ['YOUNG', 'WEST'], ['ECAB', 'METcateg', 'WEST'], ['METcateg', 'YOUNG', 'WEST'], ['GROW', 'YOUNG', 'WEST'], ['YOUNG', 'OLD', 'WEST'], ['ECAB', 'METcateg', 'YOUNG', 'OLD']]
[0.47608367732087664, 0.5310092765756806, 0.3439860168584413, 0.2481628259703127, 0.28465899522914073, 0.6024122003866281, 0.43500479378787593, 0.3537950875398891, 0.3496945256759534, 0.5206355471256685]


From the function, we can see that using predictors ECAB, METcateg, and WEST produces the model with the highest $R^2$ where all of the predictor's p-values are less than .05. Now we just need to see the full summary for that model.

In [798]:
X = expDFmod[['ECAB', 'METcateg', 'WEST']]
X = sm.add_constant(X)
y = expDFmod['EX']
red_model = sm.OLS(y, X).fit()
red_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     EX   R-squared:                       0.628
Model:                            OLS   Adj. R-squared:                  0.602
Method:                 Least Squares   F-statistic:                     24.74
Date:                Fri, 11 Dec 2020   Prob (F-statistic):           1.56e-09
Time:                        19:17:07   Log-Likelihood:                -239.44
No. Observations:                  48   AIC:                             486.9
Df Residuals:                      44   BIC:                             494.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         71.1863     26.599      2.676      0.010      17.579     124.793
ECAB           1.4257      0.259      5.500      0.000       0.903       1.948
METcateg      23.6014      7.832      3.014      0.004       7.818      39.385
WEST          41.9348     10.721      3.911      0.000      20.327      63.542
==============================================================================
Omnibus:                        0.264   Durbin-Watson:                   2.620
Prob(Omnibus):                  0.876   Jarque-Bera (JB):                0.042
Skew:                           0.072   Prob(JB):                        0.979
Kurtosis:                       3.021   Cond. No.                         498.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The reduced model is written out as such:

$$
\texttt{EX}=71.19+1.43\texttt{(ECAB)}+23.60\texttt{(METcateg)}+41.93\texttt{(WEST)}
$$

###  1G) Compare Your Models
Perform a hypothesis test to determine if the predictors removed from the full model from part 1E) to create the model in 1F) should be kept in the model.  Provide the hypothesis, perform the test, and state the conclusions using p-values.  Be sure to provide your answer in terms of the original problem, and interpret the parameters.

We use a partial F-test to determine if the paramaters removed from the model in part 1E were in fact not adding useful information to the model. This test will determine if the sum of squared errors in the full model is significantly lower than the sum of squared errors in the reduced model. Our null hypothesis is that the coefficients of GROW = YOUNG = OLD = 0 and the alternative is that at least one of these predictors is significantly different from zero. 

In [799]:
#YOUR CODE FOR 1G HERE
dfd = 41 # d.o.f. of full model
dfn = 3 # number of removed predictors
F_alph = stats.f.ppf(.95, dfn=dfn, dfd=dfd) # F-stat rejection region
sse_red = 0 # sum of squared errors for the reduced model
sse_full = 0 # sum of squared errors for the full model

# calculate sum of squared errors for each model
for i in range(0, 48):
    sse_red += red_model.resid[i]**2
    sse_full += model_mod.resid[i]**2

F_stat = ((sse_red-sse_full)/dfn)/(sse_full/dfd)
print("F-test result: %f" % F_stat)
print("F_alpha: %f" % F_alph)

F-test result: 0.693865
F_alpha: 2.832747


We fail to reject the null hypothesis that using the predictors GROW, YOUNG, or OLD significantly improved the fit of the model versus when they were excluded, using a p-value of .05. That is to say the reduced model successfully threw out useless predictors. 

### 1H) Explore A Different Model
Your model in 1F should still include at least some of the levels of MET.  Suppose we removed these, and instead only included the *other* terms. 

- First, make a plot of the residuals of this no-MET model (y) against the *continuous* values of MET (x) from the original problem.
- Before running any code, describe what kinds of *continuous* nonlinear functions or transformations might have captured this type of response.
- Then, implement a polynomial regression capturing the effect of MET while including the same non-MET terms as you did in 2f.
- Plot the residuals of the polynomial regression multiple linear model against the continuous predictor MET, and argue why you've included enough polynomial terms to satisfy any relevant modeling assumptions.

In [800]:
#YOUR CODE FOR 1H HERE
red_model_noMET = sm.OLS(expDFmod['EX'], expDFmod[['ECAB', 'WEST']]).fit()
sse_noMET = [y for y in red_model_noMET.resid**2]
val_MET = list(expDF['MET'])

plt.scatter(val_MET, sse_noMET)
plt.xlabel("Continuous values of MET")
plt.ylabel("Residuals of no-MET reduced model")
plt.show()

A 2-degree polynomial would likely fit this data best. It is easier to see the non-linear relationship between the continuous values of MET and the residuals of the model when the model includes ALL the other predictors. 

In [801]:
expDF['METsq'] = expDF['MET']**2

y, X = dmatrices('EX ~ ECAB + MET + WEST + METsq', data=expDF, return_type='dataframe')
model_poly = sm.OLS(y,X).fit()
model_poly.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     EX   R-squared:                       0.664
Model:                            OLS   Adj. R-squared:                  0.632
Method:                 Least Squares   F-statistic:                     21.22
Date:                Fri, 11 Dec 2020   Prob (F-statistic):           1.02e-09
Time:                        19:18:03   Log-Likelihood:                -237.00
No. Observations:                  48   AIC:                             484.0
Df Residuals:                      43   BIC:                             493.4
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    163.0454     28.880      5.646      0.000     104.803     221.288
ECAB           1.5542      0.275      5.650      0.000       0.999       2.109
MET           -2.8698      0.757     -3.789      0.000      -4.397      -1.342
WEST          38.4597     11.189      3.437      0.001      15.896      61.023
METsq          0.0304      0.009      3.541      0.001       0.013       0.048
==============================================================================
Omnibus:                        1.830   Durbin-Watson:                   2.604
Prob(Omnibus):                  0.401   Jarque-Bera (JB):                1.236
Skew:                          -0.093   Prob(JB):                        0.539
Kurtosis:                       2.236   Cond. No.                     2.10e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.1e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

$$
\texttt{EX}=163.05+1.55\texttt{(ECAB)}-2.87\texttt{(MET)}+0.03\texttt{(MET)}^2+38.46\texttt{(WEST)}
$$

In [802]:
sse = [y for y in model_poly.resid**2]

plt.scatter(val_MET, sse)
plt.xlabel("Continuous values of MET")
plt.ylabel("Residuals of no-MET reduced model")
plt.show()

The plot of residuals for the polynomial regression vs the values for MET is much less heteroskedastic than the plot that used the linear model. Adding more polynomial terms caused the beta value for the higher order predictors to plummet, and didn't add capture any more variability as measured by adjusted $R^2$. 


### 1I) Favorite Model
Between all the models above (full model, backwards selection, continuous MET) pick your favorite, and interpret all the parameters.  Why is this model your favorite?  Be sure to provide interpretations in terms of the original problem, including the original scale of the dependent and independent variables.

My favorite model was the one created in part 1H. This is despite all three models capturing a similar amount of variance in the data. Compared to the full model, this one discards many of the predictors that were obviously collinear, even in an intuitive sense, such as YOUNG and MET. Also, using a version of MET that wasn't abstracted, as it was in the first reduced model, should provide a more accurate prediction. Using the polynomial regression instead of the linear one also produced greater confidence in each of the predictors used, as indicated by lower p-values across the board as well as tighter confidence intervals for the same significance level. In conclusion, the polynomial multiple linear regression model used in this problem still wasn't able to capture the variance in state and local expenditure to the standard we normally expect. But, the final model used the imperfect predictors it had in the best manner possible. 

## Problem 2: Amazon Forest Fires
Now that you have shown yourself to be an expert in understanding and managing forest fire risk, a non-profit trying to protect the amazon rain forest has recruited you to join their data science corps. For your first task, they've given you a dataset (`amazon.csv`) with the number of reported forest fires in each state in the Amazon region of Brazil during each month between 1998 and 2017. The Brazilian government has 500 extra wildland firefighters and they have asked your non-profit to determine which state or states they should allocate these firefighters to during each month of the year. To do this, they want you to calculate an 80% confidence interval for the mean and median number of fires that occur during each month for each state, and use those statistics to determine where the firefighters should be assigned.

### A) Loading The CSV
Read the csv located in `amazon.csv` into a pandas data frame. Brazil and many other countries use the period (.) symbol as a thousands separator and a comma (,) as the decimal separator. Ex. One Thousand And $\frac{75}{100}$ would be represented as $1.000,75$ instead of the familiar english notation $1,000.75$. When you read it in, you'll need to use a period(.) as the thousands separator and a comma(,) as the decimal separator. Because the comma is already in use as the decimal separator, this file uses a different character to separate columns in the data. Open up the file in a text editor and figure out what character was used. Then find the correct arguments to `pd.read_csv` to read in this file properly. Look up the docs if you're unsure what the arguments you'll need are. Print out the `.info` summary of the dataframe after you've read it in.

In [561]:
#2A CODE HERE
firesDF = pd.read_csv('amazon.csv', sep=';', thousands='.', decimal=',')
firesDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6454 entries, 0 to 6453
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   year    6453 non-null   object 
 1   state   6452 non-null   object 
 2   month   6454 non-null   object 
 3   number  6448 non-null   float64
 4   date    6450 non-null   object 
dtypes: float64(1), object(4)
memory usage: 252.2+ KB


### Part B: Data Cleaning

This dataset isn't paticularly useful in it's current state, so we'll need to clean it up a bit. Some data scientists say that most of their job is to wrangle data, so this will give you a taste of cleaning a real world data set. Perform the following tasks. 
1. Drop the 'date' column. The only information this column holds is the year, which we already have in another column. Use the `.info` summary provided to check your work.
2. Drop any rows with null values in any of the remaining columns. Use the provided code to print the number of rows remaining after this step.
3. Print all the unique values of the 'month' column. You'll notice that one is encoded with a differant character encoding then the format that pandas is using.
4. Convert the Portugese month names to English month names. If you'd like to use them, we've included the 'month_name' and the 'different_encoding' modules of the python calendar library. There are many ways to accomplish this task, and these modules are not required, but may make things easier. As part of this step, you should make sure that the Portugese month with the encoding problem is translated to the correct english month. Use the `.unique` method provided for you to check your work. 
5. Check the number column for any values that seem impossible. If you find any values you think are impossible, drop them. As a guidline, we would never expect a single state to have more than 50,000 reported forest fires in a single month. Also keep in mind that we are tracking forest fires here. Do negative or fractional forest fires really make sense? You should check for any obivously impossible conditions that you think might occur, and drop rows accordingly. Use the provided code to print the number of rows remaining after this step.
6. Since you're new on the job, some of your co-workers may have played a prank on you... Print out all the unique values of the 'year' column and drop any rows with values that don't make sense. Use the provided code to print the number of rows remaining after this step.
7. For every state in the data, print the number of rows the state has associated with it. A number of states have far more observations than the others. Each state should have roughly 240 observations (20 years multiplied by 12 months/year minus any bad data). Drop all the observations for any states that have more than 240 rows associated with them.
    2. For two points of extra credit, figure out why these states have way more rows associated with them than they should. If you choose to do the extra credit, put your answer in the markdown cell below. 
8. To give you an idea of whether your answer is correct, we've provided a unit test below the last cell. It should pass. If it doesn't, go back and figure out which step has gone awry.

We've given you a code cell for each task to make organizing the grading a bit easier. Please perform step 1 in the first code cell and so on.

**NOTE:** Since some of these tasks are not totally trivial, you may use any resources other than your classmates on this part of this problem. This means you may consult google, stack overflow, the python/pandas documentation, some random book on pandas you might have, etc... But you may only work with your partner for help, no other classmates. We will also be more helpful on this problem in office hours and in response to your *private* piazza messages.  ***CITE ALL RESOURCES USED IN A CODE COMMENT. A URL OR A BOOK TITLE IS SUFFICIENT. ANY CODE OBIVOUSLY COPIED FROM OUTSIDE SOURCES WITH OUT A CITATION WILL EARN YOU NO CREDIT ON THIS PROBLEM.***

In [562]:
#2B Step 1 CODE HERE
amazon_fires = firesDF.drop(['date'], axis=1)
amazon_fires.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6454 entries, 0 to 6453
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   year    6453 non-null   object 
 1   state   6452 non-null   object 
 2   month   6454 non-null   object 
 3   number  6448 non-null   float64
dtypes: float64(1), object(3)
memory usage: 201.8+ KB


In [563]:
#2B Step 2 CODE HERE
amazon_fires = amazon_fires.dropna(axis=0, how='any')
print(len(amazon_fires))

6446


In [564]:
#2B Step 3 CODE HERE
print(amazon_fires['month'].unique())

['Janeiro' 'Fevereiro' 'Marï¿½o' 'Abril' 'Maio' 'Junho' 'Julho' 'Agosto'
 'Setembro' 'Outubro' 'Novembro' 'Dezembro']


In [565]:
#2B Step 4 CODE HERE
for row in range(0, len(amazon_fires['month'])):
    if(amazon_fires['month'].iloc[row] == 'Janeiro'):
        amazon_fires['month'].iloc[row] = 'January'
    elif(amazon_fires['month'].iloc[row] == 'Fevereiro'):
        amazon_fires['month'].iloc[row] = 'February'
    elif(amazon_fires['month'].iloc[row] == 'Abril'):
        amazon_fires['month'].iloc[row] = 'April'
    elif(amazon_fires['month'].iloc[row] == 'Maio'):
        amazon_fires['month'].iloc[row] = 'May'
    elif(amazon_fires['month'].iloc[row] == 'Junho'):
        amazon_fires['month'].iloc[row] = 'June'
    elif(amazon_fires['month'].iloc[row] == 'Julho'):
        amazon_fires['month'].iloc[row] = 'July'
    elif(amazon_fires['month'].iloc[row] == 'Agosto'):
        amazon_fires['month'].iloc[row] = 'August'
    elif(amazon_fires['month'].iloc[row] == 'Setembro'):
        amazon_fires['month'].iloc[row] = 'September'
    elif(amazon_fires['month'].iloc[row] == 'Outubro'):
        amazon_fires['month'].iloc[row] = 'October'
    elif(amazon_fires['month'].iloc[row] == 'Novembro'):
        amazon_fires['month'].iloc[row] = 'November'
    elif(amazon_fires['month'].iloc[row] == 'Dezembro'):
        amazon_fires['month'].iloc[row] = 'December'
    elif((amazon_fires['month'].iloc[row]).startswith('Mar')):
        amazon_fires['month'].iloc[row] = 'March'

print(amazon_fires['month'].unique())
print(len(amazon_fires))

C:\Users\lysso\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


['January' 'February' 'March' 'April' 'May' 'June' 'July' 'August'
 'September' 'October' 'November' 'December']
6446


In [566]:
print(amazon_fires['state'].unique())

['Acre' 'Alagoas' 'Amapa' 'Amazonas' 'Bahia' 'Ceara' 'Distrito Federal'
 'Espirito Santo' 'Goias' 'Maranhao' 'Mato Grosso' 'Minas Gerais' 'Para'
 'Paraiba' 'Pernambuco' 'Piau' 'Rio' 'Rondonia' 'Roraima' 'Santa Catarina'
 'Sao Paulo' 'Sergipe' 'Tocantins']


In [567]:
#2B Step 5 CODE HERE
amazon_fires = amazon_fires[amazon_fires.number <= 50000]
amazon_fires = amazon_fires[amazon_fires.number >= 0]

df_len = len(amazon_fires['number'])
bad_rows = []
for row in range(0, df_len-2):
    if((amazon_fires['number'].iloc[row]).is_integer() == False): # tests if value is a whole number
        bad_rows.append(row)

fix=0
for i in range(0, len(bad_rows)): 
    amazon_fires = amazon_fires.drop(bad_rows[i]-fix, axis=0)
    fix +=1
    
print(bad_rows)
print(len(amazon_fires))

[1863, 2072, 5573]
6441


In [462]:
years = ['1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007',
         '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017']

In [568]:
#2B Step 6 CODE HERE
amazon_fires = amazon_fires[amazon_fires.year != '1000bc']
amazon_fires = amazon_fires[amazon_fires.year != '-40']
amazon_fires = amazon_fires[amazon_fires.year != '10bc']
amazon_fires = amazon_fires[amazon_fires.year != 'our new data scientist won\'t notice this']
print(amazon_fires['year'].unique())
print(len(amazon_fires))

['1998' '1999' '2000' '2001' '2002' '2003' '2004' '2005' '2006' '2007'
 '2008' '2009' '2010' '2011' '2012' '2013' '2014' '2015' '2016' '2017']
6437


In [569]:
#2B Step 7 CODE HERE
# print the number of rows associated with each state
state_count = amazon_fires['state'].value_counts()
print(state_count)
amazon_fires = amazon_fires[amazon_fires.state != 'Rio']
amazon_fires = amazon_fires[amazon_fires.state != 'Mato Grosso']
amazon_fires = amazon_fires[amazon_fires.state != 'Paraiba']

Rio                 715
Mato Grosso         476
Paraiba             474
Alagoas             240
Distrito Federal    239
Piau                239
Pernambuco          239
Amapa               239
Rondonia            239
Amazonas            239
Sergipe             239
Maranhao            239
Acre                239
Sao Paulo           239
Bahia               239
Roraima             239
Ceara               239
Tocantins           239
Santa Catarina      238
Goias               238
Espirito Santo      238
Minas Gerais        237
Para                235
Name: state, dtype: int64


In [570]:
print(len(amazon_fires))

4772


In [571]:
#2B UNIT TEST
assert \
    len(amazon_fires['state'].unique()) == 20 and \
    list(amazon_fires['month'].unique()) == \
        ['January', 'February', 'March', 'April', 'May', 'June',
             'July', 'August','September', 'October', 'November',
             'December'] and \
    len(amazon_fires) == 4772, 'something is wrong in part B.'

<PROBLEM 2B STEP 7.B. EXTRA CREDIT RESPONSE HERE>

### Part C: Medians and Means!
In this part of the problem, we'll calculate an 80% confidence interval for both the mean and median number of wildfires each state has during each month of the year. 

For the mean you should use the appropriate confidence interval with the correct distribution. Remember to check how many observations we have. Use the sample standard deviation. 

For the median, we'll have to bootstrap it because the median is not known to be normally distributed. You should bootstrap 1000 samples of the same length as the original sample for each month for each state. Calculate the median for each bootstrapped sample. Then take the middle 80% of the bootstrapped medians as your confidnce interval. This is called a bootstrapped percentile median. There are a few more complex and slightly more rigourous ways to estimate the median from bootstrapped samples, but this will serve for our purposes.

You're given a dictionary of dictionaries to store your confidence intervals for the medians and means in. 

Take a look at the dictionary structure below. 

The idea here is that for every month, for every state, you will fill in the `mean_CI` with a length two list that contains the low and high end of the confidence interval for the true mean number of fires for that state in that month. 

Similiarly, for every month, for every state, you will fill in the `median_CI` with a length two list that contains the low and high end of the confidence interval for the true median number of fires for that state in that month.

For example:

When you're done `months['January']['Acre']['mean_CI']` should be a list with the low and high bounds for the confidence interval of the true mean number of wildfires in the state of Acre in January. So `months['January']['Acre']['mean_CI'][0]` should be the low end of the CI for the mean, and `months['January']['Acre']['mean_CI'][1]` should be the high end of the CI for the mean.

`months['January']['Acre']['median_CI']` should hold the confidence interval for the true median number of wildfires in the state of Acre in January. So `months['January']['Acre']['median_CI'][0]` should be the low end of the CI for the median, and `months['January']['Acre']['median_CI'][1]` should be the high end of the CI for the median.

In [767]:
#GIVEN CODE DO NOT CHANGE THIS!!!
#YOU SHOULD BE WRITING CODE IN THE NEXT CELL(s) THAT FILLS IN THE 'months' DICTIONARY.

#If you're curious what copy and deep copy do and why we used them here see an explanation 
#here: https://thispointer.com/python-how-to-copy-a-dictionary-shallow-copy-vs-deep-copy/

from copy import deepcopy

mean_median_dict = {
    'mean_CI' : None,
    'median_CI': None
}

CI_median_num_fires = {
    'Acre': dict(mean_median_dict),
    'Alagoas':dict( mean_median_dict),
    'Amapa':dict( mean_median_dict),
    'Amazonas':dict( mean_median_dict),
    'Bahia':dict( mean_median_dict),
    'Ceara':dict( mean_median_dict),
    'Distrito Federal':dict( mean_median_dict),
    'Espirito Santo':dict( mean_median_dict),
    'Goias':dict( mean_median_dict),
    'Maranhao':dict( mean_median_dict),
    'Minas Gerais':dict( mean_median_dict),
    'Para':dict( mean_median_dict),
    'Pernambuco':dict( mean_median_dict),
    'Piau':dict( mean_median_dict),
    'Rondonia':dict( mean_median_dict),
    'Roraima':dict( mean_median_dict),
    'Santa Catarina':dict( mean_median_dict),
    'Sao Paulo':dict( mean_median_dict),
    'Sergipe':dict( mean_median_dict),
    'Tocantins':dict( mean_median_dict)  
}

months = {
    'January': deepcopy(CI_median_num_fires),
    'February': deepcopy(CI_median_num_fires),
    'March': deepcopy(CI_median_num_fires), 
    'April': deepcopy(CI_median_num_fires), 
    'May': deepcopy(CI_median_num_fires),
    'June': deepcopy(CI_median_num_fires),
    'July': deepcopy(CI_median_num_fires),
    'August': deepcopy(CI_median_num_fires), 
    'September': deepcopy(CI_median_num_fires), 
    'October': deepcopy(CI_median_num_fires),
    'November': deepcopy(CI_median_num_fires),
    'December': deepcopy(CI_median_num_fires)
}

In [768]:
# using the same data structure to collect data for (state, month) pairs
observations = {
    'samples' : [],
    'std_dev': None,
    'num_samples' : None,
    'boot_medians': None
}

states_dict = {
    'Acre': deepcopy(observations),
    'Alagoas':deepcopy( observations),
    'Amapa':deepcopy( observations),
    'Amazonas':deepcopy( observations),
    'Bahia':deepcopy( observations),
    'Ceara':deepcopy( observations),
    'Distrito Federal':deepcopy( observations),
    'Espirito Santo':deepcopy( observations),
    'Goias':deepcopy( observations),
    'Maranhao':deepcopy( observations),
    'Minas Gerais':deepcopy( observations),
    'Para':deepcopy( observations),
    'Pernambuco':deepcopy( observations),
    'Piau':deepcopy( observations),
    'Rondonia':deepcopy( observations),
    'Roraima':deepcopy( observations),
    'Santa Catarina':deepcopy( observations),
    'Sao Paulo':deepcopy( observations),
    'Sergipe':deepcopy( observations),
    'Tocantins':deepcopy( observations)  
}

data = {
    'January': deepcopy(states_dict),
    'February': deepcopy(states_dict),
    'March': deepcopy(states_dict), 
    'April': deepcopy(states_dict), 
    'May': deepcopy(states_dict),
    'June': deepcopy(states_dict),
    'July': deepcopy(states_dict),
    'August': deepcopy(states_dict), 
    'September': deepcopy(states_dict), 
    'October': deepcopy(states_dict),
    'November': deepcopy(states_dict),
    'December': deepcopy(states_dict)
}

In [769]:
#2C YOUR CODE HERE
size = len(amazon_fires)
months_list = amazon_fires['month'].unique()
states_list = amazon_fires['state'].unique()
mean_CI = [-1, -1]

# run through the dataframe and put each value in the appropriate (state, month) array
for idx in amazon_fires.index:
    month = amazon_fires['month'][idx]
    state = amazon_fires['state'][idx]
    number = amazon_fires['number'][idx]
    
    data[month][state]['samples'].append(number)
    
# go back through and count the number of samples for each (state, month) and calculate the std. dev.
for month in months_list:
    for state in states_list:
        # fill in my data dictionary with the parameters for the CI
        data[month][state]['num_samples'] = n = len(data[month][state]['samples'])
        data[month][state]['std_dev'] = std_dev = np.std(data[month][state]['samples'], ddof=1)
        x_bar = np.mean(data[month][state]['samples'])
        t_crit = stats.t.ppf(.9, n-1)
        
        # fill in the months dictionary with the CI
        mean_L, mean_U = x_bar-t_crit*(std_dev/np.sqrt(n)), x_bar+t_crit*(std_dev/np.sqrt(n))
        mean_CI = [mean_L, mean_U]
        months[month][state]['mean_CI'] = mean_CI # figure out how to properly load this
        
        # bootstrap 1000 samples and grab the median
        sample = data[month][state]['samples']
        data[month][state]['boot_medians'] = b = np.array([np.median(np.random.choice(sample, replace=True, size=n)) for ii in range(1000)])
        
        # use the middle 80% of the bootstrapped samples as the CI
        L, U = np.percentile(b, 10), np.percentile(b, 90)
        median_CI = [L, U]
        months[month][state]['median_CI'] = median_CI
        print(median_CI)

[0.0, 0.0]
[29.0, 49.0]
[2.0, 7.0]
[17.0, 45.0]
[132.0, 225.0]
[95.5, 207.0]
[0.0, 0.0]
[3.5, 11.0]
[23.0, 36.0]
[166.5, 279.55]
[37.0, 80.0]
[167.0, 420.0]
[49.0, 124.0]
[57.0, 85.0]
[5.5, 23.0]
[108.0, 302.0]
[14.0, 31.0]
[27.900000000000006, 40.5]
[25.0, 37.5]
[34.5, 46.0]
[0.0, 0.5]
[18.0, 29.5]
[0.0, 1.0]
[20.950000000000003, 55.5]
[122.0, 200.0]
[9.0, 19.5]
[0.0, 1.0]
[7.5, 16.0]
[25.0, 49.0]
[16.5, 46.5]
[66.5, 111.0]
[26.450000000000003, 56.0]
[21.0, 30.5]
[16.5, 24.5]
[7.0, 9.5]
[130.0, 260.2000000000003]
[9.0, 19.0]
[56.0, 87.0]
[19.0, 33.0]
[11.0, 29.5]
[0.0, 1.0]
[25.5, 32.0]
[0.0, 1.0]
[11.0, 47.5]
[138.0, 178.5]
[3.0, 7.5]
[0.0, 0.0]
[4.5, 11.0]
[29.0, 53.0]
[16.0, 32.0]
[69.0, 105.0]
[14.0, 27.0]
[12.0, 32.0]
[14.5, 27.0]
[9.0, 15.0]
[177.5, 417.0]
[14.0, 35.0]
[70.5, 84.0]
[22.950000000000003, 41.0]
[16.0, 58.0]
[0.0, 1.0]
[5.0, 16.0]
[0.0, 0.0]
[3.5, 12.0]
[76.0, 129.5]
[2.0, 4.0]
[0.0, 1.0]
[2.5, 8.0]
[34.0, 71.0]
[20.0, 62.0]
[60.0, 99.0]
[9.0, 20.0]
[2.0, 7.0]
[9.5,

In [770]:
#DONT CHANGE THIS. WE USE IT TO MAKE THE OUTPUT LEGIBLE FOR GRADING
import pprint
pp = pprint.PrettyPrinter(indent=1)
pp.pprint(months)

{'April': {'Acre': {'mean_CI': [0.7631702958582414, 3.336829704141758],
                    'median_CI': [0.0, 1.0]},
           'Alagoas': {'mean_CI': [7.979189204540319, 16.52081079545968],
                       'median_CI': [5.0, 16.0]},
           'Amapa': {'mean_CI': [0.3412383897376437, 0.9587616102623564],
                     'median_CI': [0.0, 0.0]},
           'Amazonas': {'mean_CI': [6.9737040719879255, 12.226295928012075],
                        'median_CI': [3.5, 12.0]},
           'Bahia': {'mean_CI': [93.8962665362685, 157.1037334637315],
                     'median_CI': [76.0, 129.5]},
           'Ceara': {'mean_CI': [2.695004965166694, 4.904995034833306],
                     'median_CI': [2.0, 4.0]},
           'Distrito Federal': {'mean_CI': [0.6266818228169977,
                                            1.8733181771830023],
                                'median_CI': [0.0, 1.0]},
           'Espirito Santo': {'mean_CI': [7.3942600245658525,
                    

                            'median_CI': [2931.0, 5638.0]},
               'Roraima': {'mean_CI': [25.355752693135237, 44.44424730686476],
                           'median_CI': [14.5, 44.0]},
               'Santa Catarina': {'mean_CI': [251.0820000393573,
                                              447.61799996064275],
                                  'median_CI': [172.0, 416.0]},
               'Sao Paulo': {'mean_CI': [585.5569773036776, 946.3430226963225],
                             'median_CI': [495.0, 809.0]},
               'Sergipe': {'mean_CI': [0.45808539321873587, 1.1419146067812642],
                           'median_CI': [0.0, 1.0]},
               'Tocantins': {'mean_CI': [3847.0546098158884, 5331.045390184112],
                             'median_CI': [3330.0, 5149.0]}}}


In [771]:
#Given Test for the mean confidence intervals

rounded_mean_CI = [round(x, 2) for x in months['April']['Acre']['mean_CI']]
assert rounded_mean_CI == [0.76, 3.34], 'somethings wrong in the mean'

In [772]:
#Given test for the median confidence intervals. 
#Your code is probably correct if it passes this test, but since bootstrapping the medain is a stochastic process
#you may have this test fail. If it fails, run it a few times. 
#If it continues to fail, your code is probably incorrect.

low_median_CI = months['April']['Acre']['median_CI'][0]
high_median_CI = months['April']['Acre']['median_CI'][1]
assert -1 <= low_median_CI <= 1 and 0 <= high_median_CI <= 3, 'somethings wrong in the median'

### Part E: Where Do The Firefighters Go?
Now, we'll determine which state the Brazilian government should assign it's fire fighters to. For each month of the year, you should perform the folllowing selection process:
1. Find the state with the highest CI for the median for this month (it's easiest and ok to just use the upper bound here). 
2. Find any states that have a median CI that overlaps with the highest CI foud in step 1. If no states overlap with the highest CI found in step 1, then use that state. 
3. If overlapping confidence intervals are found on the median, we'll use the CI for the mean to break ties.
4. Out of the states with overlapping CIs for median (every state in part 3), find the state with the highest mean CI. 
5. Determine if any of the states from part 3 have a mean CI that overlaps with the state found in step 4. 
6. If no state overlap with the state found in part 4, then just use that state. If other states have overlapping mean CIs too, then we'll split up the firefighters and assign some of them to every state that has both an overlapping median and mean CI with the state that has the highest median CI.

Once you've used the selection process above, use a markdown table to display a list of each state that recieves  some of the firefighters for each month.

In [789]:
#2E YOUR CODE HERE

# used to hold final assignments of firefighters to states
assignments = {
    'January': None,
    'February': None,
    'March': None, 
    'April': None, 
    'May': None,
    'June': None,
    'July': None,
    'August': None, 
    'September': None, 
    'October': None,
    'November': None,
    'December': None
}

def assign_firefighters(month):
    med_CI_highest = 0 # highest upper bound for median CI
    mean_CI_highest = 0 # highest upper bound for mean CI of the states whose median CIs overlap with the highest
    highest_median_state = "" # state with highest upper bound for median CI
    highest_mean_state = "" # state with highest upper bound mean CI of potential_states
    potential_states = [-1] # states whose median CI overlaps with the one with the highest upper bound
    final_states = [-1] # states which get returned if there is overlap
    #print("month: %s" % month)
    # find the state with the highest upper-bound on the median CI
    for state in states_list:
        if(months[month][state]['median_CI'][1] > med_CI_highest):
            med_CI_highest = months[month][state]['median_CI'][1]
            highest_median_state = potential_states[0] = state
    #print("highest median state: %s" % highest_median_state)
    # find all the states whose median CI overlaps with the median CI of the state found above
    for state in states_list:
        if(months[month][state]['median_CI'][1] > months[month][highest_median_state]['median_CI'][0] and \
        state != highest_median_state):
            potential_states.append(state)
    #print("overlapping median states: %s" % potential_states)
    # return if no overlapping median CIs
    if(len(potential_states) == 1):
        #print("\n")
        return potential_states[0]

    # of all the states chosen thus far, find the one with the highest upper bound on the mean CI
    for state in potential_states:
        if(months[month][state]['mean_CI'][1] > mean_CI_highest):
            mean_CI_highest = months[month][state]['mean_CI'][1]
            highest_mean_state = final_states[0] = state
    #print("highest mean state: %s" % highest_mean_state)
    # now see if the mean CIs of those states overlap with the highest of the bunch
    for state in potential_states:
        if(months[month][state]['mean_CI'][1] > months[month][highest_mean_state]['mean_CI'][0] and \
        state != highest_mean_state):
            final_states.append(state)
    #print("overlapping mean states: %s\n" % final_states)
    return final_states
        
for month in months_list:
    assignments[month] = assign_firefighters(month)
print(assignments)

{'January': ['Para', 'Roraima'], 'February': ['Roraima'], 'March': ['Roraima'], 'April': ['Bahia', 'Roraima', 'Minas Gerais', 'Sao Paulo', 'Tocantins'], 'May': ['Tocantins'], 'June': ['Tocantins'], 'July': ['Maranhao', 'Tocantins'], 'August': 'Para', 'September': ['Para', 'Rondonia'], 'October': ['Para', 'Maranhao'], 'November': ['Para'], 'December': ['Para']}


| Month     | States to send firefighters                               |
|-----------|-----------------------------------------------------------|
| January   | Para, Roraima                                             |
| February  | Roraima                                                   |
| March     | Roraima                                                   |
| April     | Roraima, Bahia, Goias, Minas Gerais, Sao Paolo, Tocantins |
| May       | Tocantins                                                 |
| June      | Tocantins                                                 |
| July      | Maranhao, Tocantins                                       |
| August    | Para                                                      |
| September | Para, Rondonia                                            |
| October   | Para, Maranhao                                            |
| November  | Para                                                      |
| December  | Para
